In [600]:
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
pd.options.display.max_rows = 4000

In [601]:
(train_X,train_y),(test_X,test_y)=mnist.load_data()

In [602]:
train_X.shape

(60000, 28, 28)

In [603]:
train_X = train_X/255.0
test_X = test_X/255.0

In [604]:
train_y

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [605]:
train_y,test_y=to_categorical(train_y,10),to_categorical(test_y,10)

In [607]:
train_X=train_X.reshape(-1,28,28,1)
train_X=train_X.reshape(-1,28,28,1)


In [608]:
train_X

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [609]:
class Conv2D:
    def __init__(self,num_filters,filter_size):
        self.num_filters=num_filters
        self.filter_size=filter_size
        self.filters=np.random.randn(num_filters,filter_size,filter_size)/filter_size**2
        
    def iterate_regions(self,image):
        h,w=image.shape
        for i in range(h-self.filter_size+1):
            for j in range(w-self.filter_size+1):
                yield image[i:i+self.filter_size,j:j+self.filter_size],i,j
        
    
    def forward(self,input):
        self.last_input=input
        h,w,_=input.shape
        output_h=h-self.filter_size+1
        output_w=w-self.filter_size+1
        output=np.zeros((output_h,output_w,self.num_filters))
        for region,i,j in self.iterate_regions(input[:,:,0]):
            output[i,j]=np.sum(region*self.filters,axis=(1,2))
        return output
    
    def backward(self, d_L_d_out, learn_rate):
        d_L_d_filters = np.zeros(self.filters.shape)
        h, w = self.last_input.shape[:2]

        for region, i, j in self.iterate_regions(self.last_input[:, :, 0]):
            for f in range(self.num_filters):
                d_L_d_filters[f] += d_L_d_out[i, j, f] * region

        # Update filters
        self.filters -= learn_rate * d_L_d_filters
        return None

In [610]:
class MaxPool2D:
    
    def iterate_regions(self,image):
        h,w,num_filters=image.shape
        for i in range(0,h,2):
            for j in range(0,w,2):
                yield image[i:i+2,j:j+2],i//2,j//2
    
    
    def forward(self,input):
        self.last_input=input
        h,w,num_filters=input.shape
        output=np.zeros((h//2,w//2,num_filters))
        for region,i,j in self.iterate_regions(input):
            output[i,j]=np.max(region,axis=(0,1))
        return output
    
    def backward(self, d_L_d_out):
        d_L_d_input = np.zeros(self.last_input.shape)
        for region, i, j in self.iterate_regions(self.last_input):
            h, w, f = region.shape
            amax = np.max(region, axis=(0, 1))

            for i2 in range(h):
                for j2 in range(w):
                    for f2 in range(f):
                        if region[i2, j2, f2] == amax[f2]:
                            d_L_d_input[i*2+i2, j*2+j2, f2] = d_L_d_out[i, j, f2]
        return d_L_d_input

In [611]:
class Softmax:
    def forward(self,input):
        self.last_input_shape=input.shape
        input=input.flatten()
        self.last_input=input
        exp=np.exp(input)
        self.out=exp/np.sum(exp,axis=0)
        return self.out        
    
    def backward(self,d_L_d_out,learn_rate):
        d_out_d_t=np.diagflat(self.out)-np.outer(self.out,self.out)
        d_L_d_t=np.dot(d_out_d_t,d_L_d_out)
        return d_L_d_t.reshape(self.last_input_shape)

In [612]:
class Dense:
    
    def __init__(self,input_size,output_size):
        self.weights=np.random.randn(input_size,output_size)
        self.biases=np.zeros(output_size)
        
    def forward(self,input):
        self.last_input_shape=input.shape
        self.last_input=input.flatten()
        output=np.dot(self.last_input,self.weights)+self.biases
        return output
    
    def backward(self, d_L_d_out, learn_rate):
        d_L_d_out = d_L_d_out.reshape(1, -1)  # Ensure d_L_d_out is a 2D array
        d_L_d_weights = np.dot(self.last_input.reshape(-1, 1), d_L_d_out)  # Reshape self.last_input to 2D
        self.weights -= learn_rate * d_L_d_weights
        self.biases -= learn_rate * np.sum(d_L_d_out, axis=0)

        return np.dot(d_L_d_out, self.weights.T).reshape(self.last_input_shape)
        
        

In [613]:
conv=Conv2D(8,3)
pool=MaxPool2D()
softmax=Softmax()
dense=Dense(13*13*8,10)

In [614]:
def forward(image,label):
    out=conv.forward(image)
    out=pool.forward(out)
    out=dense.forward(out)

    out=softmax.forward(out)
    loss=-np.log(out[label])
    acc=1 if np.argmax(out)==label else 0

    return out,loss,acc

In [615]:
def train(im,label,learn_rate=0.005):
    out,loss,acc=forward(im,label)
    grad=np.zeros(10)
    grad[label]=-1/out[label]
    
    grad = softmax.backward(grad, learn_rate)
    grad = dense.backward(grad, learn_rate)
    grad = pool.backward(grad)
    conv.backward(grad, learn_rate)
    
    return loss,acc

In [ ]:
for epoch in range(3): 
    print(f"--- Epoch {epoch+1} ---")
    loss = 0
    num_correct = 0

    for i, (im, label) in enumerate(zip(train_X, train_y)):
        if i % 1000 == 0:
            print(f"{i} steps completed")

        l, acc = train(im, np.argmax(label))
        loss += l
        num_correct += acc

    print(f"Epoch {epoch+1} | Loss: {loss} | Accuracy: {num_correct / len(train_X)}")

print("--- Testing ---")
loss = 0
num_correct = 0
for im, label in zip(test_X, test_y):
    _, l, acc = forward(im, np.argmax(label))
    loss += l
    num_correct += acc

print(f"Test Loss: {loss} | Test Accuracy: {num_correct / len(test_X)}")
